In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pivot-table2-csv/pivot_table.csv


In [ ]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
import pandas as pd
import time
import gc

# Učitaj podatke
df_pivot = pd.read_csv('/kaggle/input/pivot-table2-csv/pivot_table.csv', index_col=0)
df_melt = df_pivot.stack().reset_index().rename(columns={'level_1': 'Movie_Id', 0: 'Rating'})

# Definiši Reader objekat i učitaj podatke
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_melt[['Cust_Id', 'Movie_Id', 'Rating']], reader)

# Podeli podatke na obuku i test
trainset, testset = train_test_split(data, test_size=0.25)

# Parametri za testiranje
n_factors_values = [10, 20, 30]
n_epochs_values = [10, 20, 30]
lr_all_values = [0.002, 0.005, 0.01]
reg_all_values = [0.02, 0.1, 0.2]

for n_factors in n_factors_values:
    for n_epochs in n_epochs_values:
        for lr_all in lr_all_values:
            for reg_all in reg_all_values:
                # Postavi parametre za SVD
                algo = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all)
                
                # Treniraj model
                start_time = time.time()
                algo.fit(trainset)
                end_time = time.time()
                print(f"Parametri: n_factors={n_factors}, n_epochs={n_epochs}, lr_all={lr_all}, reg_all={reg_all}")
                print(f"Vreme fitovanja: {end_time - start_time:.2f} sekundi")
                
                # Predviđaj na test setu
                start_time = time.time()
                predictions = algo.test(testset)
                end_time = time.time()
                print(f"Vreme predviđanja: {end_time - start_time:.2f} sekundi")
                
                # Izmeri preciznost modela
                rmse = accuracy.rmse(predictions, verbose=False)
                mae = accuracy.mae(predictions, verbose=False)
                print(f'RMSE: {rmse}, MAE: {mae}')
                
                # Oslobodi memoriju
                del algo
                gc.collect()

                print('-' * 50)


Parametri: n_factors=10, n_epochs=10, lr_all=0.002, reg_all=0.02
Vreme fitovanja: 68.57 sekundi
Vreme predviđanja: 67.18 sekundi
RMSE: 0.9109300960553707, MAE: 0.7171153446531995
--------------------------------------------------
Parametri: n_factors=10, n_epochs=10, lr_all=0.002, reg_all=0.1
Vreme fitovanja: 60.50 sekundi
Vreme predviđanja: 43.69 sekundi
RMSE: 0.9155503581857476, MAE: 0.723429955183873
--------------------------------------------------
Parametri: n_factors=10, n_epochs=10, lr_all=0.002, reg_all=0.2
Vreme fitovanja: 60.60 sekundi
Vreme predviđanja: 42.87 sekundi
RMSE: 0.9189599456588383, MAE: 0.7290093061192171
--------------------------------------------------
Parametri: n_factors=10, n_epochs=10, lr_all=0.005, reg_all=0.02
Vreme fitovanja: 60.68 sekundi
Vreme predviđanja: 43.39 sekundi
RMSE: 0.8789624637018913, MAE: 0.6866886739545666
--------------------------------------------------
Parametri: n_factors=10, n_epochs=10, lr_all=0.005, reg_all=0.1
Vreme fitovanja: 60